# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [4]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [5]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4, test_size=0.2)

### Fit a Model

Fit a linear regression model on the training set

In [6]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [7]:
# Instantiate and fit a linear regression model
linreg = LinearRegression()

linreg.fit(X_train, y_train)

LinearRegression()

### Calculate MSE

Calculate the mean squared error on the test set

In [8]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

In [9]:
# Calculate MSE on test set

test_mse = mean_squared_error(y_test, linreg.predict(X_test))
print(test_mse)

0.15233997210708142


## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [10]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score


In [22]:
# Find MSE scores for a 5-fold cross-validation
mse_score = cross_val_score(linreg, X, y, cv = 5, scoring="neg_mean_squared_error")
mse_score

array([-0.12431546, -0.19350065, -0.1891053 , -0.17079325, -0.20742705])

In [23]:
# Get the average MSE score

ave_mse_score = mse_score.mean()
ave_mse_score

-0.17702834210001095

Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [13]:
# Your answer here


## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [24]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [32]:
def kfolds(data, k):
    
    """
    Splits a dataset into `k` folds, ensuring that if the dataset size isn't divisible by `k`,
    the first few folds are one larger than the later ones.
    
    Parameters:
    data (pd.DataFrame): The dataset to split.
    k (int): The number of folds.
    
    Returns:
    list: A list of `k` DataFrames, each representing a fold.
    """
    
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Determine the sizes of the folds
    n = len(data)
    fold_sizes = [n // k + (1 if i < n % k else 0) for i in range(k)]
    folds = []
    
    # Your code here
    start = 0
    for size in fold_sizes:
        end = start + size
        folds.append(data.iloc[start:end])
        start = end
        
    
    return folds

In [33]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange
2  indigo 

    color
3  yellow
4    blue 

    color
5   green
6  violet 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [37]:
X_folds = kfolds(X, 5)
y_folds = kfolds(y, 5)

### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd

# Initialize test error list and number of folds
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = pd.concat([X_folds[i] for i in range(k) if i != n])  # All folds except nth
    X_test = X_folds[n]  # nth fold as test
    y_train = pd.concat([y_folds[i] for i in range(k) if i != n])  # All folds except nth
    y_test = y_folds[n]  # nth fold as test
    
    # Fit a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Evaluate test errors
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    test_errs.append(mse)

print(test_errs)


[0.17595331097301345, 0.15660997502632573, 0.22060406223365156, 0.17864474046470116, 0.1666495643807151]


In [39]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Perform manual cross-validation
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = pd.concat([X_folds[i] for i in range(k) if i != n])
    X_test = X_folds[n]
    y_train = pd.concat([y_folds[i] for i in range(k) if i != n])
    y_test = y_folds[n]
    
    # Fit a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Evaluate test errors
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    test_errs.append(mse)

# Average test error from manual cross-validation
avg_manual_mse = np.mean(test_errs)

# Perform cross-validation using scikit-learn
model = LinearRegression()
sklearn_mse_scores = cross_val_score(model, X, y, cv=k, scoring="neg_mean_squared_error")
avg_sklearn_mse = -np.mean(sklearn_mse_scores)

# Print and compare results
print(f"Manual Cross-Validation Errors: {test_errs}")
print(f"Average Manual MSE: {avg_manual_mse}")
print(f"Scikit-learn Cross-Validation MSE Scores: {-sklearn_mse_scores}")
print(f"Average Scikit-learn MSE: {avg_sklearn_mse}")


Manual Cross-Validation Errors: [0.17595331097301345, 0.15660997502632573, 0.22060406223365156, 0.17864474046470116, 0.1666495643807151]
Average Manual MSE: 0.1796923306156814
Scikit-learn Cross-Validation MSE Scores: [0.12431546 0.19350065 0.1891053  0.17079325 0.20742705]
Average Scikit-learn MSE: 0.17702834210001095


In [18]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = None
    X_test = None
    y_train = None
    y_test = None
    
    # Fit a linear regression model
    None
    
    # Evaluate test errors
    None

print(test_errs)

[]


If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [19]:
# Compare your results with sklearn results


This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.